In [1]:
workers = 2
batch_size=50
nz = 100
n_epoch = 20
lr = 0.0002
beta1 = 0.5
outf = './result-GAN'
display_interval = 600

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision.datasets import MNIST
from torchvision import transforms
import torchvision.utils as vutils
import numpy as np
from matplotlib import pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

batch_size = 256
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)), transforms.Lambda(lambda x: x.view(-1))])
mnist_train = MNIST("MNIST",train=True, download=True, transform=transform)

dataloader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)

cpu


In [3]:
class Generator(nn.Module):
    def __init__(self, nz=100):
        """
        :param nz: 入力ベクトルzの次元
        :param nch_g: 最終層の入力チャネル数
        :param nch: 出力画像のチャネル数
        """
        super().__init__()
        self.layers = nn.Sequential(
                nn.Linear(nz, 256),                      
                nn.ReLU(),                          
                nn.Linear(256, 512),
                nn.ReLU(),
                nn.Linear(512, 28*28),
                nn.Tanh()
        )

    def forward(self, z):
        return self.layers(z)
    
    
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
                nn.Linear(28*28, 384),   
                nn.LeakyReLU(negative_slope=0.2),
                nn.Linear(384, 128),
                nn.LeakyReLU(negative_slope=0.2),
                nn.Linear(128, 1),
                nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [10]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(batch_size, nz, device=device)

netG = Generator(nz=nz).to(device)
netD = Discriminator().to(device)


optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)  # 識別器D用
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)  # 生成器G用

In [11]:
G_losses = []
D_losses = []
D_x_out = []
D_G_z1_out = []

# 学習のループ
for epoch in range(n_epoch):
    for itr, data in enumerate(dataloader):
        real_image = data[0].to(device)
        sample_size = real_image.size(0)
        
        # 標準正規分布からノイズを生成
        noise = torch.randn(sample_size, nz, device=device)
        # 本物画像に対する識別信号の目標値「1」
        real_target = torch.full((sample_size,), 1., device=device)
        # 生成画像に対する識別信号の目標値「0」
        fake_target = torch.full((sample_size,), 0., device=device) 
        
        ############################
        # 識別器Dの更新
        ###########################
        netD.zero_grad()    # 勾配の初期化

        output = netD(real_image)   # 識別器Dで本物画像に対する識別信号を出力
        errD_real = criterion(output.squeeze(), real_target)  # 本物画像に対する識別信号の損失値
        D_x = output.mean().item()  # 本物画像の識別信号の平均

        fake_image = netG(noise)    # 生成器Gでノイズから生成画像を生成
        
        output = netD(fake_image.detach())  # 識別器Dで本物画像に対する識別信号を出力
        errD_fake = criterion(output.squeeze(), fake_target)  # 生成画像に対する識別信号の損失値
        D_G_z1 = output.mean().item()  # 生成画像の識別信号の平均

        errD = errD_real + errD_fake    # 識別器Dの全体の損失
        errD.backward()    # 誤差逆伝播
        optimizerD.step()   # Dのパラメーターを更新

        ############################
        # 生成器Gの更新
        ###########################
        netG.zero_grad()    # 勾配の初期化
        
        output = netD(fake_image)   # 更新した識別器Dで改めて生成画像に対する識別信号を出力
        errG = criterion(output.squeeze(), real_target)   # 生成器Gの損失値。Dに生成画像を本物画像と誤認させたいため目標値は「1」
        errG.backward()     # 誤差逆伝播
        D_G_z2 = output.mean().item()  # 更新した識別器Dによる生成画像の識別信号の平均

        optimizerG.step()   # Gのパラメータを更新

        if itr % display_interval == 0: 
            print('[{}/{}][{}/{}] Loss_D: {:.3f} Loss_G: {:.3f} D(x): {:.3f} D(G(z)): {:.3f}/{:.3f}'
                  .format(epoch + 1, n_epoch,
                          itr + 1, len(dataloader),
                          errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        if epoch == 0 and itr == 0:     # 初回に本物画像を保存する
            vutils.save_image(real_image, '{}/real_samples.png'.format(outf),
                              normalize=True, nrow=10)

        # ログ出力用データの保存
        D_losses.append(errD.item())
        G_losses.append(errG.item())
        D_x_out.append(D_x)
        D_G_z1_out.append(D_G_z1)

    ############################
    # 確認用画像の生成
    ############################
    fake_image = netG(fixed_noise)  # 1エポック終了ごとに確認用の生成画像を生成する
    fake_image = fake_image.reshape(batch_size, 1, 28, 28)
    vutils.save_image(fake_image.detach(), '{}/fake_samples_epoch_{:03d}.png'.format(outf, epoch + 1),
                      normalize=True, nrow=10)


[1/20][1/235] Loss_D: 1.366 Loss_G: 0.666 D(x): 0.525 D(G(z)): 0.514/0.514
[2/20][1/235] Loss_D: 1.273 Loss_G: 0.889 D(x): 0.598 D(G(z)): 0.528/0.411
[3/20][1/235] Loss_D: 0.711 Loss_G: 1.554 D(x): 0.757 D(G(z)): 0.339/0.212
[4/20][1/235] Loss_D: 0.638 Loss_G: 2.465 D(x): 0.833 D(G(z)): 0.348/0.089
[5/20][1/235] Loss_D: 1.015 Loss_G: 3.345 D(x): 0.941 D(G(z)): 0.603/0.036
[6/20][1/235] Loss_D: 0.726 Loss_G: 3.522 D(x): 0.581 D(G(z)): 0.020/0.048


KeyboardInterrupt: 